# MS2DeepScore API – Quickstart

*In order to use omigami, you need an access token provided by DataRevenue.*
## Introduction

This is a short guide on how you can add GNPS library matches to your mass spectra by their Tanimoto Score - using Omigami's MS2DeepScore API.

The API uses a MS2DeepScore model which was trained on > 100,000 MS/MS spectra from GNPS. It prepares the input spectra and using a Siamese neural network predicts the molecular structural similarity (Tanimoto score) of the input spectra with the GNPS library spectra. It then returns you the top library matches for each of your spectra.

This notebook shows you how to:
1. Specify any MGF file with two spectra.
2. Run a library search through the MS2DeepScore API
3. Save the results as XLSX.

# 1. Load a MS/MS dataset (MGF format)

We'll use a small MS/MS dataset in the MGF format from [here](https://gnps-external.ucsd.edu/gnpslibrary/GNPS-COLLECTIONS-MISC.mgf).
You can also select any other from the [GNPS spectral library](https://gnps-external.ucsd.edu/gnpslibrary), or - of course - use your own.

*Note that you will need the Precursor_MZ field `PEPMASS` and the abundance pairs in your MGF file.*

In [1]:
# Load your own MS/MS dataset
path_to_mgf <- 'GNPS-2-SPECTRA.mgf'

# 2. Query for the best matches with MS2DeepScore
`Romigami` is a R wrapper which instantiates a virtual environment on which to install and call the Python `Omigami` package


`MS2DeepScore` is a python wrapper which:
- Builds a json payload from the MGF file
- Calls the MS2DeepScores API
- Formats the prediction results into readable dataframes

____
`match_spectra_from_path()` takes the following arguments:
- `n_best` sets the number of matches you'd like per spectrum (it is set to 10 by default).
- `include_metadata` selects the metadata fields that will be displayed in the resulting top matches.
- `ion_mode` selects the ionization mode of the measured spectra (positive or negative).

In the results dataframes, the input spectra can be identified by their number in the dataframes index, which refers to their order in the MGF file.  
*i.e.* `matches of spectrum 1` gives the spectrum_id and MS2DeepScore scores of the library spectra matches, for the first spectrum in the MGF file.

For each spectrum in the MGF file, the library spectra matches are sorted according to their MS2DeepScore similarity score (best is first).   
The following information about the predicted spectra are returned :
- `score`, the MS2DeepScore similarity score between the input spectrum and the library spectrum
- `matches of spectrum #`, the spectrum_ID of the matched library spectra for the spectrum number # in the MGF file
- `metadata`, metadata fields chosen with `include_metadata` in the input

In [2]:
# install devtools and romigami if not already installed
if("devtools" %in% installed.packages() == FALSE) {install.packages("devtools")}
devtools::install_github("omigami/romigami")

# import the romigami package
library("romigami")

In [3]:
# Initialize omigami environment
omigami <- omigami_init()

In [4]:
# Initialize MS2DeepScore Client
client <- omigami$MS2DeepScore(token="MY_TOKEN")

In [5]:
# Run MS2DeepScore Match Spectra from path 

spectra_matches <- client$match_spectra_from_path(
    mgf_path = path_to_mgf, 
    n_best = 10, 
    include_metadata = list("Smiles", "Compound_name"), 
    ion_mode = "positive"
)

## 2.1 View results
A list of dataframes is returned. To look at a specific dataframe you can call:
```r
spectra_matches[[i]]  # 'i' refers to the ith spectrum in the MGF file input
```

In [6]:
spectra_matches[[1]]

,score,compound_name,smiles
,<list>,<chr>,<chr>
CCMSLIB00004720008,0.895566,Triacetylresveratrol,CC(=O)Oc1ccc(/C=C/c2cc(OC(C)=O)cc(OC(C)=O)c2)cc1
CCMSLIB00005749695,0.8445965,19S-Methoxytubotaiwine,COC(C)C1C2N3CCC22C(NC4=CC=CC=C24)=C(C1CC3)C(=O)OC
CCMSLIB00004720014,0.8401476,Triacetylresveratrol,CC(=O)Oc1ccc(/C=C/c2cc(OC(C)=O)cc(OC(C)=O)c2)cc1
CCMSLIB00005740410,0.8212357,Humantenine,CON1C(=O)C2(CC3C4COC2CC4C(CN3C)=CC)C2=CC=CC=C12
CCMSLIB00004702935,0.8185643,Undulatoside A,Cc1cc(=O)c2c(O)cc(O[C@@H]3O[C@H](CO)[C@@H](O)[C@H](O)[C@H]3O)cc2o1
CCMSLIB00004708920,0.7970317,"2-(3,5-dihydroxy-1,4-dioxo-1,4-dihydronaphthalen-2-yl)-3-methoxy-5-methylbenzoic acid",COc1cc(C)cc(C(=O)O)c1C1=C(O)C(=O)c2c(O)cccc2C1=O
CCMSLIB00004720104,0.795813,Undulatoside A,Cc1cc(=O)c2c(O)cc(O[C@@H]3O[C@H](CO)[C@@H](O)[C@H](O)[C@H]3O)cc2o1
CCMSLIB00004702936,0.7947617,Undulatoside A,Cc1cc(=O)c2c(O)cc(O[C@@H]3O[C@H](CO)[C@@H](O)[C@H](O)[C@H]3O)cc2o1
CCMSLIB00004708921,0.7928906,"2-(3,5-dihydroxy-1,4-dioxo-1,4-dihydronaphthalen-2-yl)-3-methoxy-5-methylbenzoic acid",COc1cc(C)cc(C(=O)O)c1C1=C(O)C(=O)c2c(O)cccc2C1=O


# 3. Save results

Execute the following cell to save the results in a CSV file. For readability, each dataframe is saved in its own CSV file under the `matches` directory.

In [7]:
if (!dir.exists("matches")){
    dir.create("matches")
}
for (i in seq_along(spectra_matches)){
    matches <- data.frame(lapply(spectra_matches[i], as.character), stringsAsFactors=FALSE)
    write.csv(matches, sprintf("matches/spectrum_%s.csv", i))
}

# 4. Create Plots

The following cells will show you an example of how to visualize your matches.
## Creating a Plot of the molecular structure

In [ ]:
plotter <- omigami$MoleculePlotter()
plotter$plot_molecule_structure_grid(spectra_matches=spectra_matches[[1]], representation="smiles", draw_indices=1, molecule_image_size=list(200, 200), substructure_highlight=1)

## Creating a Plot using the ClassyFire API

In [ ]:
plotter$plot_classyfire_result(spectra_matches=spectra_matches[[1]])

## Creating a Plot using the NP-Classifier API

In [ ]:
plotter$plot_NPclassifier_result(spectra_matches=spectra_matches[[1]], color="orange")

____